In [1]:
# importing packages
import sqlite3
import pandas as pd

In [6]:
con = sqlite3.connect('./Europe_Footy_data/european_database.sqlite')
cursor_obj = con.cursor()

In [10]:
%%bash

sqlite3 './Europe_Footy_data/european_database.sqlite'

.schema

CREATE TABLE IF NOT EXISTS "divisions" (
"division" TEXT,
  "name" TEXT,
  "country" TEXT
);
CREATE TABLE IF NOT EXISTS "matchs" (
"Div" TEXT,
  "Date" DATE,
  "HomeTeam" TEXT,
  "AwayTeam" TEXT,
  "FTHG" REAL,
  "FTAG" REAL,
  "FTR" TEXT,
  "season" INTEGER
);


In [18]:
whole_df = pd.read_sql('''
        
            SELECT  TRIM(LOWER(name)) as division_name, 
                    TRIM(LOWER(country)) as country, 
                    TRIM(LOWER(HomeTeam)) as hometeam, 
                    TRIM(LOWER(AwayTeam)) as awayteam,
                    CAST(FTHG as int) as home_regulation_goals,
                    CAST(FTAG as int) as away_regulation_goals,
                    FTR as result,
                    Date as matchdate,
                    season
            FROM divisions as d
            JOIN matchs as m
            ON d.division = m.Div
                        ''', con)
whole_df

,division_name,country,hometeam,awayteam,home_regulation_goals,away_regulation_goals,result,matchdate,season
0,division 1a,belgium,club brugge,mouscron,2,0,H,2005-08-05,2006
1,division 1a,belgium,anderlecht,louvieroise,6,0,H,2005-08-06,2006
2,division 1a,belgium,beveren,fc brussels,0,2,A,2005-08-06,2006
3,division 1a,belgium,gent,waregem,1,3,A,2005-08-06,2006
4,division 1a,belgium,lokeren,genk,1,1,D,2005-08-06,2006
...,...,...,...,...,...,...,...,...,...
120632,süper lig,turkey,karagumruk,denizlispor,5,1,H,2021-05-15,2021
120633,süper lig,turkey,kayserispor,fenerbahce,1,2,A,2021-05-15,2021
120634,süper lig,turkey,rizespor,buyuksehyr,0,2,A,2021-05-15,2021
120635,süper lig,turkey,sivasspor,kasimpasa,2,1,H,2021-05-15,2021


In [27]:
whole_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120637 entries, 0 to 120636
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   division_name          120637 non-null  object
 1   country                120637 non-null  object
 2   hometeam               120637 non-null  object
 3   awayteam               120637 non-null  object
 4   home_regulation_goals  120637 non-null  int64 
 5   away_regulation_goals  120637 non-null  int64 
 6   result                 120637 non-null  object
 7   matchdate              120637 non-null  object
 8   season                 120637 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 8.3+ MB


What are the most goals scored in a match in the premier league for each season? Choose the top 5 seasons and order by max_goals and season ascending 

In [41]:
prem_max_goals = pd.read_sql ('''
            SELECT  season, MAX(home_regulation_goals + away_regulation_goals) as max_goals
            FROM(SELECT TRIM(LOWER(name)) as division_name, 
                        TRIM(LOWER(country)) as country, 
                        TRIM(LOWER(HomeTeam)) as hometeam, 
                        TRIM(LOWER(AwayTeam)) as awayteam,
                        CAST(FTHG as int) as home_regulation_goals,
                        CAST(FTAG as int) as away_regulation_goals,
                        FTR as result,
                        Date as matchdate,
                        season
                    FROM divisions as d
                    JOIN matchs as m
                    ON d.division = m.Div) as sub
            WHERE division_name = 'premier league'
            GROUP BY season
            ORDER BY max_goals DESC, season asc
            LIMIT 5
        
            
            
''',con)
prem_max_goals

,season,max_goals
0,2008,11
1,2010,10
2,2012,10
3,2013,10
4,2014,9


How much does homefield advantage matter? Find the difference in the french leagues

In [85]:
france_win_differential = pd.read_sql('''

SELECT division_name, season, 
    ROUND(SUM((CASE WHEN result = 'H' THEN 1 ELSE 0 END) * 100.00)/ COUNT(*), 2) as home_winning_percentage,
    ROUND(SUM((CASE WHEN result = 'A' THEN 1 ELSE 0 END) * 100.00)/ COUNT(*), 2) as away_winning_percentage,
    ROUND(SUM((CASE WHEN result = 'H' THEN 1 ELSE 0 END) * 100.00)/ COUNT(*), 2) -
    ROUND(SUM((CASE WHEN result = 'A' THEN 1 ELSE 0 END) * 100.00)/ COUNT(*), 2) as differential
    
FROM(SELECT TRIM(LOWER(name)) as division_name, 
            TRIM(LOWER(country)) as country, 
            TRIM(LOWER(HomeTeam)) as hometeam, 
            TRIM(LOWER(AwayTeam)) as awayteam,
            CAST(FTHG as int) as home_regulation_goals,
            CAST(FTAG as int) as away_regulation_goals,
            FTR as result,
            Date as matchdate,
            season
    FROM divisions as d
    JOIN matchs as m
    ON d.division = m.Div) as sub
WHERE country = 'france'
GROUP BY division_name, season
ORDER BY 1, differential DESC 
                    
                    ''', con)
france_win_differntial

,division_name,season,home_winning_percentage,away_winning_percentage,differential
0,ligue 1,2007,47.63,21.58,26.05
1,ligue 1,2012,47.11,24.47,22.64
2,ligue 1,2017,48.95,26.32,22.63
3,ligue 1,2020,48.03,26.88,21.15
4,ligue 1,2006,44.74,24.21,20.53
5,ligue 1,2010,47.11,27.37,19.74
6,ligue 1,2015,47.63,29.21,18.42
7,ligue 1,2008,43.95,25.53,18.42
8,ligue 1,2013,44.74,26.84,17.90
9,ligue 1,2011,41.32,24.47,16.85


How many goals does Liverpool average over the seasons?

In [117]:
Antwerp_averages = pd.read_sql('''

WITH home_goal as (SELECT season, hometeam, COUNT(*) as total_games, SUM(home_regulation_goals) as home_goals
FROM(SELECT TRIM(LOWER(name)) as division_name, 
            TRIM(LOWER(country)) as country, 
            TRIM(LOWER(HomeTeam)) as hometeam, 
            TRIM(LOWER(AwayTeam)) as awayteam,
            FTHG as home_regulation_goals,
            FTAG as away_regulation_goals,
            FTR as result,
            Date as matchdate,
            season
    FROM divisions as d
    JOIN matchs as m
    ON d.division = m.Div) as sub1
WHERE hometeam = 'liverpool'
GROUP BY season, hometeam),

away_goal AS (SELECT season, awayteam, COUNT(*) as total_games, SUM(away_regulation_goals) as away_goals
FROM(SELECT TRIM(LOWER(name)) as division_name, 
            TRIM(LOWER(country)) as country, 
            TRIM(LOWER(HomeTeam)) as hometeam, 
            TRIM(LOWER(AwayTeam)) as awayteam,
            FTHG as home_regulation_goals,
            FTAG as away_regulation_goals,
            FTR as result,
            Date as matchdate,
            season
    FROM divisions as d
    JOIN matchs as m
    ON d.division = m.Div) as sub2
WHERE awayteam = 'liverpool'
GROUP BY season, awayteam)

SELECT h.season, awayteam as team, (away_goals+home_goals)/(h.total_games + a.total_games) AS average
FROM home_goal h
JOIN away_goal a
on h.season = a.season
GROUP BY h.season





''', con)
Antwerp_averages

,season,team,average
0,2006,liverpool,1.500000
1,2007,liverpool,1.500000
2,2008,liverpool,1.763158
3,2009,liverpool,2.026316
4,2010,liverpool,1.605263
5,2011,liverpool,1.552632
6,2012,liverpool,1.236842
7,2013,liverpool,1.868421
8,2014,liverpool,2.657895
9,2015,liverpool,1.368421


In [86]:
%%bash

sqlite3 './Europe_Footy_data/european_database.sqlite'

.schema

CREATE TABLE IF NOT EXISTS "divisions" (
"division" TEXT,
  "name" TEXT,
  "country" TEXT
);
CREATE TABLE IF NOT EXISTS "matchs" (
"Div" TEXT,
  "Date" DATE,
  "HomeTeam" TEXT,
  "AwayTeam" TEXT,
  "FTHG" REAL,
  "FTAG" REAL,
  "FTR" TEXT,
  "season" INTEGER
);


How long is the season duration in the 2008 season across all leagues? Is there any difference?

In [143]:
season_duration = pd.read_sql ('''
    SELECT name as divname, COUNT(*) number_games_played, Julianday(Max(date)) - Julianday(MIN(date)) as lengthofseasonindays, 
    DENSE_RANK() OVER(ORDER BY Julianday(Max(date)) - Julianday(MIN(date)) DESC) as rankonlen
    FROM matchs
    JOIN divisions
    ON matchs.Div = divisions.division
    WHERE season = 2008
    GROUP BY div
    
''', con)
season_duration

,divname,number_games_played,lengthofseasonindays,rank
0,LaLiga 2,462,295.0,1
1,Ligue 2,380,294.0,2
2,Scottish Premiership,228,292.0,3
3,Ligue 1,380,287.0,4
4,2. Bundesliga,306,282.0,5
5,Seria B,462,281.0,6
6,Bundesliga,306,281.0,6
7,Division 1A,306,281.0,6
8,Süper Lig,306,274.0,7
9,Premier League,380,274.0,7
